In [30]:
import numpy as np
import random

In [388]:
KEY_TIE = 'tie'
KEY_ONGOING = 'ongoing'
KEY_WON_P1 = 'won_p1'
KEY_WON_P2 = 'won_p2'

In [364]:
def update_state(state, q_table, epsilon, player):
    r = random.random()
    possible_actions = np.array([(idx, value) for (idx, value) in enumerate(q_table.flatten()) if not np.isnan(value) ])
    if len(possible_actions) < 1:
        print('ERROR: all fields already filled, game ended')
        return

    if r < (1-epsilon):
        # find max of q table
        action_max = np.nanmax(possible_actions[:,1])
        max_action_idxs = [p_action[0] for p_action in possible_actions if p_action[1] == action_max]
        if len(max_action_idxs) > 1:
            # take random when max at several positions
            action_idx = random.choice(max_action_idxs)
        else:
            action_idx = max_action_idxs[0]

    else:
        action_idx = random.choice(possible_actions[:,0])

    i = int(action_idx/3)
    j = int(action_idx % 3)
    state[i, j] = player
    q_table[i, j] = np.nan

    return state

In [434]:
def game_status(state):
    # check if tie
    if np.count_nonzero(state==0) == 0:
        return KEY_TIE
    # check if someone has won
    possible_winning_comb = np.zeros((8, 3))
    possible_winning_comb[:3,:] = state
    possible_winning_comb[3:6,:] = state.T
    possible_winning_comb[6,:] = state.diagonal()
    possible_winning_comb[7,:] = np.fliplr(state).diagonal()
    for row in possible_winning_comb:
        if abs(row).all() == 1:
            if row[0] == 1: return KEY_WON_P1
            if row[1] == -1: return KEY_WON_P2

    return KEY_ONGOING

In [ ]:
def update_q_table(q_table):
    

In [437]:
game_status(state)

'won_p2'

In [384]:
state = np.zeros((3,3))
q_table = np.zeros((3,3))

In [436]:
update_state(state, q_table, 1.0, -1)

array([[-1.,  0., -1.],
       [ 0., -1.,  0.],
       [-1., -1.,  0.]])